<left><img src="https://github.com/pandas-dev/pandas/raw/main/web/pandas/static/img/pandas.svg" alt="pandas Logo" style="width: 200px;"/></left>
<right><img src="https://matplotlib.org/stable/_images/sphx_glr_logos2_003.png" style="width: 200px;"/></right>

# Pandas and Matplotlib - EMODNET
---

In [33]:
import pandas as pd
import matplotlib.pyplot as plt

import datetime
from pathlib import Path


## Load ERDDAP data

 [ERDDAP](https://coastwatch.pfeg.noaa.gov/erddapinfo/) is a data server that gives you a simple, consistent way to download data in the format and the spatial and temporal coverage that you want. ERDDAP is a web application with an interface for people to use. It is also a RESTful web service that allows data access directly from any computer program (e.g. Matlab, R, or webpages)."

This notebook uses the python client [erddapy](https://pyoceans.github.io/erddapy) to help construct the RESTful URLs and translate the responses into Pandas and Xarray objects. 

A typical ERDDAP RESTful URL looks like:

[https://data.ioos.us/gliders/erddap/tabledap/whoi_406-20160902T1700.mat?depth,latitude,longitude,salinity,temperature,time&time>=2016-07-10T00:00:00Z&time<=2017-02-10T00:00:00Z &latitude>=38.0&latitude<=41.0&longitude>=-72.0&longitude<=-69.0](https://data.ioos.us/gliders/erddap/tabledap/whoi_406-20160902T1700.mat?depth,latitude,longitude,salinity,temperature,time&time>=2016-07-10T00:00:00Z&time<=2017-02-10T00:00:00Z&latitude>=38.0&latitude<=41.0&longitude>=-72.0&longitude<=-69.0)

Let's break it down to smaller parts:

- **server**: https://data.ioos.us/gliders/erddap/
- **protocol**: tabledap
- **dataset_id**: whoi_406-20160902T1700
- **response**: .mat
- **variables**: depth,latitude,longitude,temperature,time
- **constraints**:
    - time>=2016-07-10T00:00:00Z
    - time<=2017-02-10T00:00:00Z
    - latitude>=38.0
    - latitude<=41.0
    - longitude>=-72.0
    - longitude<=-69.0

### EMODNET:  
https://emodnet.ec.europa.eu/en/emodnet-web-service-documentation#non-ogc-web-services

erddap EMODNET physics:  
https://prod-erddap.emodnet-physics.eu/erddap/index.html  
https://prod-erddap.emodnet-physics.eu/erddap/tabledap/documentation.html  

### erddapy  
https://github.com/ioos/erddapy

>pip install erddapy

In [34]:
from erddapy import ERDDAP
from erddapy.core.url import urlopen

In [35]:
# ERDDAP for EMODNET Physics
server = 'https://coastwatch.pfeg.noaa.gov/erddap'
protocol = 'tabledap'
emodnet = ERDDAP(server=server, protocol=protocol)


server = 'https://prod-erddap.emodnet-physics.eu/erddap'
protocol = 'tabledap'
emodnet = ERDDAP(server=server, protocol=protocol)

In [36]:
min_time = '2010-01-01T00:00:00Z'
max_time = '2012-12-31T23:00:00Z'
min_lon, max_lon = -17, -15
min_lat, max_lat = 44.1, 44.5

In [37]:
kw = {
    'min_lon': min_lon,'max_lon': max_lon,'min_lat': min_lat,'max_lat': max_lat,
    'min_time': min_time,'max_time': max_time
}

search_url = emodnet.get_search_url(response='csv', **kw)
search_df = pd.read_csv(urlopen(search_url))
search_df = search_df[['Institution', 'Dataset ID','tabledap']]
search_df

,Institution,Dataset ID,tabledap
0,GLODAP,GLODAPv2_2021,https://prod-erddap.emodnet-physics.eu/erddap/...


In [38]:
dataset_id = 'GLODAPv2_2021'
emodnet.dataset_id = dataset_id
emodnet.response = "csv"
emodnet.constraints = {
     "time>=": min_time,
     "time<=": max_time,
    "latitude>=": min_lat,
    "latitude<=": max_lat,
    "longitude>=": min_lon,
    "longitude<=": max_lon,
}
emodnet.variables = ["longitude", "latitude", "time",
    "G2temperature", "G2salinity", "G2pressure"
]

df = emodnet.to_pandas()

HTTPError: Error {
    code=404;
    message="Not Found: Your query produced no matching results. (nRows = 0)";
}


In [ ]:
df

---

## The pandas [`DataFrame`](https://pandas.pydata.org/docs/user_guide/dsintro.html#dataframe)...
...is a **labeled**, two-dimensional columnar structure, similar to a table, spreadsheet, or the R `data.frame`.

![dataframe schematic](https://github.com/pandas-dev/pandas/raw/main/doc/source/_static/schemas/01_table_dataframe.svg "Schematic of a pandas DataFrame")

The `columns` that make up our `DataFrame` can be lists, dictionaries, NumPy arrays, pandas `Series`, or many other data types not mentioned here. Within these `columns`, you can have data values of many different data types used in Python and NumPy, including text, numbers, and dates/times. The first column of a `DataFrame`, shown in the image above in dark gray, is uniquely referred to as an `index`; this column contains information characterizing each row of our `DataFrame`. Similar to any other `column`, the `index` can label rows by text, numbers, datetime objects, and many other data types. Datetime objects are a quite popular way to label rows.

For our first example using Pandas DataFrames, we start by reading in some data in comma-separated value (`.csv`) format. We retrieve this dataset from the Pythia DATASETS class (imported at the top of this page); however, the dataset was originally contained within the NCDC teleconnections database. This dataset contains many types of geoscientific data, including El Nino/Southern Oscillation indices. For more information on this dataset, review the description [here](https://www.ncdc.noaa.gov/teleconnections/enso/indicators/sst/).

In [ ]:
df

In [ ]:
# Set index
df.set_index(pd.to_datetime(df['time (UTC)']), inplace=True)

In [ ]:
df

In [ ]:
df.index[0]

### Read file

In [ ]:
p_file = Path('__file__').resolve()
dir_data = p_file.parents[0] / 'data'

fnd = dir_data / 'GLODAPv2.2021.csv'
df2 = pd.read_table(fnd, sep=',')
df2

,G2cruise,G2region,G2station,G2cast,G2year,G2month,G2day,G2hour,G2minute,G2latitude,...,G2toc,G2tocf,G2doc,G2docf,G2don,G2donf,G2tdn,G2tdnf,G2chla,G2chlaf
0,1,4,319,1,1984,7,20,14,46,80.567,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
1,1,4,319,1,1984,7,20,14,46,80.567,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
2,1,4,319,1,1984,7,20,14,46,80.567,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
3,1,4,319,1,1984,7,20,14,46,80.567,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
4,1,4,319,1,1984,7,20,14,46,80.567,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,1,4,324,1,1984,7,22,2,48,81.197,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
85,1,4,324,1,1984,7,22,2,48,81.197,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
86,1,4,324,1,1984,7,22,2,48,81.197,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
87,1,4,324,1,1984,7,22,2,48,81.197,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9


The `DataFrame` index, as described above, contains information characterizing rows; each row has a unique ID value, which is displayed in the index column.  By default, the IDs for rows in a `DataFrame` are represented as sequential integers, which start at 0.

In [ ]:
df.index

NameError: name 'df' is not defined

At the moment, the index column of our `DataFrame` is not very helpful for humans. However, Pandas has clever ways to make index columns more human-readable. The next example demonstrates how to use optional keyword arguments to convert `DataFrame` index IDs to a human-friendly datetime format.

In [ ]:
# For pandas version > 2.0
# df2 = pd.read_table(fnd, sep=',', dtype={'G2year': int, 'G2month': int, 'G2day': int, 
#                                                        'G2hour': int, 'G2minute': int},
#                     parse_dates={'time': ['G2year', 'G2month', 'G2day', 'G2hour', 'G2minute']},
#                     date_format='%Y %m %d %H %M', 
# )

df2 = pd.read_table(fnd, sep=',', dtype={'G2year': int, 'G2month': int, 'G2day': int, 
                                                       'G2hour': int, 'G2minute': int},
                    parse_dates={'time': ['G2year', 'G2month', 'G2day', 'G2hour', 'G2minute']},
                    infer_datetime_format=True, 
)
df2 
# date was not recognized!

,time,G2cruise,G2region,G2station,G2cast,G2latitude,G2longitude,G2bottomdepth,G2maxsampdepth,G2bottle,...,G2toc,G2tocf,G2doc,G2docf,G2don,G2donf,G2tdn,G2tdnf,G2chla,G2chlaf
0,1984 7 20 14 46,1,4,319,1,80.567,7.2267,733,724,1,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
1,1984 7 20 14 46,1,4,319,1,80.567,7.2267,733,724,2,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
2,1984 7 20 14 46,1,4,319,1,80.567,7.2267,733,724,3,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
3,1984 7 20 14 46,1,4,319,1,80.567,7.2267,733,724,4,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
4,1984 7 20 14 46,1,4,319,1,80.567,7.2267,733,724,5,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,1984 7 22 2 48,1,4,324,1,81.197,16.7930,1258,1252,22,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
85,1984 7 22 2 48,1,4,324,1,81.197,16.7930,1258,1252,21,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
86,1984 7 22 2 48,1,4,324,1,81.197,16.7930,1258,1252,19,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
87,1984 7 22 2 48,1,4,324,1,81.197,16.7930,1258,1252,18,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9


In [ ]:
import datetime as dt
df2 = pd.read_table(fnd, sep=',', dtype={'G2year': int, 'G2month': int, 'G2day': int, 
                                                       'G2hour': int, 'G2minute': int})
df2['time'] = pd.to_datetime(pd.to_datetime({'year':df2.G2year, 'month':df2.G2month.values, 
                            'day':df2.G2day, 'hour':df2.G2hour, 'minute':df2.G2minute})) 
df2.drop(['G2year', 'G2month', 'G2day', 'G2hour', 'G2minute'], axis=1, inplace=True)
df2.set_index('time', inplace=True)
df2


,G2cruise,G2region,G2station,G2cast,G2latitude,G2longitude,G2bottomdepth,G2maxsampdepth,G2bottle,G2pressure,...,G2toc,G2tocf,G2doc,G2docf,G2don,G2donf,G2tdn,G2tdnf,G2chla,G2chlaf
time,,,,,,,,,,,,,,,,,,,,,
1984-07-20 14:46:00,1,4,319,1,80.567,7.2267,733,724,1,8.6,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
1984-07-20 14:46:00,1,4,319,1,80.567,7.2267,733,724,2,8.6,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
1984-07-20 14:46:00,1,4,319,1,80.567,7.2267,733,724,3,48.9,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
1984-07-20 14:46:00,1,4,319,1,80.567,7.2267,733,724,4,48.9,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
1984-07-20 14:46:00,1,4,319,1,80.567,7.2267,733,724,5,148.0,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984-07-22 02:48:00,1,4,324,1,81.197,16.7930,1258,1252,22,68.0,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
1984-07-22 02:48:00,1,4,324,1,81.197,16.7930,1258,1252,21,104.0,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9
1984-07-22 02:48:00,1,4,324,1,81.197,16.7930,1258,1252,19,148.0,...,-9999,9,-9999,9,-9999,9,-9999,9,-9999,9


Each of our data rows is now helpfully labeled by a datetime-object-like index value; this means that we can now easily identify data values not only by named columns, but also by date labels on rows. This is a sneak preview of the `DatetimeIndex` functionality of Pandas; this functionality enables a large portion of Pandas' timeseries-related usage. Don't worry; `DatetimeIndex` will be discussed in full detail later on this page. In the meantime, let's look at the columns of data read in from the `.csv` file:

In [ ]:
df.columns

Index(['hs', 'tm', 'tp', 'dirm', 'dp', 'spr', 'h', 'lm', 'lp', 'uw', 'vw',
       'time'],
      dtype='object')

## The pandas [`Series`](https://pandas.pydata.org/docs/user_guide/dsintro.html#series)...

...is essentially any one of the columns of our `DataFrame`. A `Series` also includes the index column from the source `DataFrame`, in order to provide a label for each value in the `Series`.

![pandas Series](https://github.com/pandas-dev/pandas/raw/main/doc/source/_static/schemas/01_table_series.svg "Schematic of a pandas Series")

The pandas `Series` is a fast and capable 1-dimensional array of nearly any data type we could want, and it can behave very similarly to a NumPy `ndarray` or a Python `dict`. You can take a look at any of the `Series` that make up your `DataFrame`, either by using its column name and the Python `dict` notation, or by using dot-shorthand with the column name:

### Get columns informations  

df['name']  
df.name 

if name is a number  
df[145]  
df.15 is not valid!

In [ ]:
df['G2temperature']

KeyError: 'G2temperature'

<div class="alert alert-block alert-info">
<b>Tip:</b> You can also use the dot notation illustrated below to specify a column name, but this syntax is mostly provided for convenience. For the most part, this notation is interchangeable with the dictionary notation; however, if the column name is not a valid Python identifier (e.g., it starts with a number or space), you cannot use dot notation.</div>

In [ ]:
df.G2temperature

In [ ]:
df = pd.read_table('data/data_waves.dat', header=None, delim_whitespace=True, 
                   names=['YY', 'mm', 'DD', 'time', 'hs', 'tm', 'tp', 'dirm', 'dp', 'spr', 'h', 'lm', 'lp', 
                          'uw', 'vw'],
                  parse_dates=[[0, 1, 2, 3]], index_col=0)
df

,hs,tm,tp,dirm,dp,spr,h,lm,lp,uw,vw
YY_mm_DD_time,,,,,,,,,,,
1979-01-01 05:00:00,2.817,6.8,7.7,226.8,223.4,27.2,80.02,76.40,93.04,10.30,3.32
1979-01-01 06:00:00,3.407,7.4,8.5,229.8,226.5,25.6,80.02,91.55,112.80,13.27,3.68
1979-01-01 07:00:00,4.644,8.1,9.3,231.5,229.2,24.7,80.02,110.38,135.01,17.11,6.84
1979-01-01 08:00:00,5.131,8.6,9.9,230.7,225.0,25.3,80.02,122.50,152.12,16.92,6.68
1979-01-01 09:00:00,4.297,8.6,9.9,228.4,221.1,27.5,80.02,123.26,153.09,10.32,-4.45
...,...,...,...,...,...,...,...,...,...,...,...
2000-12-31 19:00:00,0.618,7.7,11.5,217.1,218.6,62.5,80.02,124.21,204.12,-5.79,-4.57
2000-12-31 20:00:00,0.579,6.8,12.1,197.1,219.6,71.1,80.02,103.68,225.18,-5.20,-4.69
2000-12-31 21:00:00,0.537,6.2,12.0,165.5,221.5,73.3,80.02,88.59,219.36,-5.06,-4.21


### Using `.iloc` and `.loc` to index

In this section, we introduce ways to access data that are preferred by Pandas over the methods listed above. When accessing by label, it is preferred to use the `.loc` method, and when accessing by index, the `.iloc` method is preferred. These methods behave similarly to the notation introduced above, but provide more speed, security, and rigor in your value selection. Using these methods can also help you avoid [chained assignment warnings](https://pandas.pydata.org/docs/user_guide/indexing.html#returning-a-view-versus-a-copy) generated by pandas.

In [ ]:
df["1982-01-01":"1982-12-01"]

In [ ]:
df.iloc[3]

In [ ]:
df.iloc[0:12]

In [ ]:
df.loc["1982-04-01"]

In [ ]:
df.loc["1982-01-01":"1982-12-01"]

The `.loc` and `.iloc` methods also allow us to pull entire rows out of a `DataFrame`, as shown in these examples:

In [ ]:
df.loc["1982-04-01"]

In [ ]:
df.loc["1982-01-01":"1982-12-01"]

### Exercise A

- Define a new dataframe with the hs, tm, dirm data
- Select the 1980-1990 data
- Get the maximum and mean data

### Exercise B

- Define a new dataframe with the tp, uw, uv data
- Select the 1990-2000 data
- Get the minimum and mean data

### Get stats on the dataset

In [ ]:

df.describe()

In [ ]:
df.max()

## Resampling, Shifting, and Windowing

In [ ]:
df['hs']

In [ ]:
df.hs[:100].plot()

In [ ]:
df.rolling('12H').mean().hs[:100].plot()

In [ ]:
dfi = df.iloc[:500]

In [ ]:
dfi.hs.resample('24H').mean().plot(style=':', linewidth=2)

In [ ]:
df.hs.resample('A').mean()

For up-sampling, ``resample()`` and ``asfreq()`` are largely equivalent, though resample has many more options available.
In this case, the default for both methods is to leave the up-sampled points empty, that is, filled with NA values.
Just as with the ``pd.fillna()`` function discussed previously, ``asfreq()`` accepts a ``method`` argument to specify how values are imputed.
Here, we will resample the business day data at a daily frequency (i.e., including weekends):

In [ ]:
annual_max = df.groupby(df.index.year).max()
annual_max

In [ ]:
index_hs_max=df.hs.groupby(df.index.year).idxmax()
index_hs_max

In [ ]:
df.hs.plot();

In [ ]:
df.hs.resample('1Y').mean().plot();

In [ ]:
## Save files
df.to_csv

<left><img src="https://images.prismic.io/coresignal-website/135e2df3-33e4-456a-adb0-73ebaa07bc88_JSON+vs+CSV.png?auto=compress%2Cformat&fit=max&q=90&w=1200&h=1499" alt="save" style="width: 500px;"/></left>



### CSV
Name, Job title  

Jane, Analyst  
Lukas, Developer

### JSON
{“name”:”Jane”,”jobTitle”:”Analyst”}

{“name”:”Lukas”,”jobTitle”:”Developer”}

### Exercise A

- With you new dataframe
- Create a 2x2 figure
- plot variables data, the 1-year resample data, a 3 month rolling month and markers for the  annual maxima: hs for top-left and tm top-right
- plot hs-dirm and tm-dirm scatter on bottom-left and bottom-right

### Exercise B

- With you new dataframe
- Create a 2x2 figure
- plot variables data, the 1-year resample data, a 3 month rolling month and markers for the  annual maxima: tp for top-left and uw top-right
- plot tp-uw and tp-vw scatter on bottom-left and bottom-right

# Seaborn

In [ ]:
import numpy as np
import seaborn as sb

In [ ]:
# facetting histograms by subsets of data
sb.set(style="darkgrid")

tips = sb.load_dataset("tips")
tips

In [ ]:
g = sb.FacetGrid(tips, row="sex", col="time", margin_titles=True)
bins = np.linspace(0, 60, 13)
g.map(plt.hist, "total_bill", color="steelblue", bins=bins, lw=0)

In [ ]:
color = sb.color_palette()[2]
g = sb.jointplot(data=tips, x="total_bill", y="tip", kind="reg",
                  xlim=(0, 60), ylim=(0, 12), color=color)

In [ ]:
color = sb.color_palette()[2]
g = sb.jointplot(data=tips, x="total_bill", y="tip", kind="hex",
                  xlim=(0, 60), ylim=(0, 12), color=color)